In [1]:
import os
import glob 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import deepdish as dd

from functools import partial

import random

import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras import layers, models
from keras.layers.core import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.utils.class_weight import compute_class_weight

In [2]:
current_dir = os.getcwd()
model_file = 'D:\Misc\KiMoRe\\results\\kimoreModel__moviT__epoch0150__ratingLoss0.3677__exerciseLoss0.2771.hdf5'

In [3]:
class WeightedCategoricalCrossentropy(tf.keras.losses.CategoricalCrossentropy):
    
    def __init__(self, cost_mat, name='weighted_categorical_crossentropy', **kwargs):
        cost_mat = np.array(cost_mat)   
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])
        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)


    def __call__(self, y_true, y_pred, sample_weight=None):
        assert sample_weight is None, "should only be derived from the cost matrix"  
        return super().__call__(
            y_true = y_true, 
            y_pred = y_pred, 
            sample_weight = get_sample_weights(y_true, y_pred, self.cost_mat),
        )


    def get_config(self):
        config = super().get_config().copy()
        config.update({'cost_mat': (self.cost_mat)})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


def get_sample_weights(y_true, y_pred, cost_m):
    num_classes = len(cost_m)

    y_pred.shape.assert_has_rank(2)
    assert(y_pred.shape[1] == num_classes)
    y_pred.shape.assert_is_compatible_with(y_true.shape)

    y_pred = K.one_hot(K.argmax(y_pred), num_classes)

    y_true_nk1 = K.expand_dims(y_true, 2)
    y_pred_n1k = K.expand_dims(y_pred, 1)
    cost_m_1kk = K.expand_dims(cost_m, 0)

    sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
    sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

    return sample_weights_n


tf.keras.losses.WeightedCategoricalCrossentropy = WeightedCategoricalCrossentropy

In [40]:
split = 0.8

In [41]:
with open("D:\Misc\KiMoRe\\kimore_filenames_90frames_45overlap_unevenRating.txt", "r") as txt_file:
    file_contents = txt_file.readlines()
kimore_filenames = file_contents[0].split(' ')[:-1]

kimore_dicts = dd.io.load('D:\Misc\KiMoRe\\kimore_dicts_90frames_45overlap_unevenRating.h5')

train_split = round(split * len(kimore_filenames))
train_files = random.sample(kimore_filenames, train_split)
val_files = [f for f in kimore_filenames if f not in train_files]

train_idx = [i for i in range(len(kimore_filenames)) if kimore_filenames[i] in train_files]
val_idx = [i for i in range(len(kimore_filenames)) if kimore_filenames[i] in val_files]


########## GET TRAIN DATA ##########

train_input_list = []
train_ratings_list = []
train_exercises_list = []

for idx in train_idx:
    train_dict = kimore_dicts[idx]
    train_input_list.append(train_dict["image"])
    train_ratings_list.append(train_dict["rating_onehot"])
    train_exercises_list.append(train_dict["exercise_onehot"])

train_inputs = np.stack(train_input_list)
train_ratings = np.stack(train_ratings_list)
train_exercises = np.stack(train_exercises_list)


########## GET VALIDATION DATA ##########

val_input_list = []
val_ratings_list = []
val_exercises_list = []

for idx in val_idx:
    val_dict = kimore_dicts[idx]
    val_input_list.append(val_dict["image"])
    val_ratings_list.append(val_dict["rating_onehot"])
    val_exercises_list.append(val_dict["exercise_onehot"])

val_inputs = np.stack(val_input_list)
val_ratings = np.stack(val_ratings_list)
val_exercises = np.stack(val_exercises_list)


###########################################################################


########## GET RATING INFORMATION ##########

rating_onehot_list = []
rating_list = []
for dictionary in kimore_dicts:
    rating_onehot_list.append(dictionary["rating_onehot"])
    rating_list.append(dictionary["rating"])

rating_onehot_ndarray = np.stack(rating_onehot_list)
rating_class_list = sorted([i-1 for i in rating_list])


########## COMPUTE CLASS WEIGHTS ##########

sklearn_class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(rating_class_list), y= rating_class_list)

w_array = np.ones((3, 3))
w_array[0, :] = sklearn_class_weights[0]
w_array[1, :] = sklearn_class_weights[1]
w_array[2, :] = sklearn_class_weights[2]


In [43]:
model = tf.keras.models.load_model(
    model_file,
    compile = True,
    custom_objects = {'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(w_array)})

model.summary()

Model: "kimore_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['image_input[0][0]']            
                                                                                                  
 flatten (Flatten)              (None, 2048)         0           ['resnet50[0][0]']               
                                                                                                  
 dense1_30_rating (Dense)       (None, 30)           61470       ['flatten[0][0]']     

In [50]:
input_layer = keras.Input(shape=(255, 255, 3), batch_size=8, name="input_1")

# Save and convert :
model.layers[0] = input_layer
model.summary()
# model.save("reshaped_model.h5")    
# coreml_model = coremltools.converters.keras.convert('reshaped_model.h5')    
# coreml_model.save('MyPredictor.mlmodel')

Model: "kimore_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 2048)         23587712    ['image_input[0][0]']            
                                                                                                  
 flatten (Flatten)              (None, 2048)         0           ['resnet50[0][0]']               
                                                                                                  
 dense1_30_rating (Dense)       (None, 30)           61470       ['flatten[0][0]']     

In [4]:
import onnx
from onnxsim import simplify

onnx_model = onnx.load("path/to/model.onnx")
model_simpified, check = simplify(onnx_model)
onnx.save(model_simpified, "path/to/simplified/model.onnx")

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (C:\Users\charl\anaconda3\lib\site-packages\google\protobuf\internal\__init__.py)